In [20]:
#| default_exp feature.users

In [21]:
#| exporti

from dataclasses import dataclass , field

import mbison.client.core as dmda

In [22]:
#| hide
import os

In [23]:
auth = dmda.DomoAuth(
    domo_instance=os.environ["DOMO_INSTANCE"],
    access_token=os.environ["DOMO_ACCESS_TOKEN"],
)
# auth.who_am_i()
auth

*** Using Access Token Auth


DomoAuth(domo_instance='datacrew-space', username=None)

In [24]:
#| export
class User_API_Exception(dmda.API_Exception):
    def __init__(self, res, message= None):

        super().__init__(res = res,message = message)

In [25]:
#| export

def get_user_by_id(user_id, auth : dmda.DomoAuth , debug_api: bool = False):

    endpoint = f"/api/content/v3/users/{user_id}"

    params = {"includeDetails": True}

    res = dmda.domo_api_request(
        auth= auth,
        endpoint=endpoint,
        request_type= "GET",
        params = params,
        debug_api= debug_api,
    )


    if not res.is_success and res.status == 404 :
        raise User_API_Exception(res, message = f"unable to find user - {user_id}")

    return res

In [26]:
get_user_by_id(user_id = 1351541814, auth =auth, debug_api = False)


ResponseGetData(response={'id': 1351541814, 'invitorUserId': 27, 'displayName': 'jae wilson', 'role': 'Admin', 'roleId': 1, 'detail': {'title': 'Staff Analytics Engineer', 'email': 'jae@datacrew.space', 'phoneNumber': '8087651506', 'pending': False, 'active': True, 'created': 1713890872, 'modified': 1713890922, 'department': 'IT/Engineering'}}, is_success=True, status=200, auth=DomoAuth(domo_instance='datacrew-space', username=None))

In [27]:
#| export

@dataclass
class DomoUser:
    auth :dmda.DomoAuth = field(repr = False)
    id: str
    display_name: str
    role_id: int
    email : str

    @classmethod
    def from_json(cls, obj, auth : dmda.DomoAuth):
        return cls(
            auth = auth, 
            id = obj['id'],
            display_name = obj['displayName'],
            role_id = obj['roleId'],
            email = obj['detail']['email']
        )
    
    @classmethod
    def get_by_id(cls, user_id, auth, debug_api : bool = False, return_raw :bool = False):
        res = get_user_by_id(user_id = user_id, auth =auth, debug_api = debug_api)

        if return_raw:
            return res
        
        return cls.from_json(obj = res.response , auth = auth)

In [28]:
DomoUser.get_by_id(user_id= 1351541814, auth = auth)

DomoUser(id=1351541814, display_name='jae wilson', role_id=1, email='jae@datacrew.space')

In [29]:
#| hide

import nbdev
nbdev.nbdev_export()